# Transportation

$$
\begin{align}
    \text{min} \quad & \sum_{i \in I}\sum_{j \in J} c_{i, j} x_{i, j} \\
    \text{s.t.} \quad & \sum_{j \in J} x_{i, j} \leq b_{i} & \forall \; i \in I \\
    \quad & \sum_{i \in I} x_{i, j} = d_{j} & \forall \; j \in J \\
    & x_{i, j} \geq 0 & \forall \;i \in I, j \in J \\
\end{align}
$$

In [1]:
# Python native modules
import json

# Third-party packages
import pandas as pd  # Tabular data
import pyomo.environ as pyo

In [2]:
# Read input file and store in local variable `input_data`
with open("input_transp.json", mode="r", encoding="utf8") as file:
    input_data = json.load(file)

In [8]:
input_data

{'availabilities': {'S1': 14, 'S2': 26, 'S3': 11},
 'demands': {'C1': 5, 'C2': 13, 'C3': 15, 'C4': 17},
 'costs': [{'from': 'S1', 'to': 'C1', 'value': 10},
  {'from': 'S1', 'to': 'C2', 'value': 5},
  {'from': 'S1', 'to': 'C3', 'value': 20},
  {'from': 'S1', 'to': 'C4', 'value': 12},
  {'from': 'S2', 'to': 'C1', 'value': 12},
  {'from': 'S2', 'to': 'C2', 'value': 7},
  {'from': 'S2', 'to': 'C3', 'value': 12},
  {'from': 'S2', 'to': 'C4', 'value': 19},
  {'from': 'S3', 'to': 'C1', 'value': 6},
  {'from': 'S3', 'to': 'C2', 'value': 12},
  {'from': 'S3', 'to': 'C3', 'value': 16},
  {'from': 'S3', 'to': 'C4', 'value': 17}]}

In [5]:
# Create local variables to store input parameters
availabilities = input_data["availabilities"]
demands = input_data["demands"]

# Dictionary of costs indexed by tuples (origin, destination)
costs = {
    (c["from"], c["to"]): c["value"]
    for c in input_data["costs"]
}

In [7]:
costs

{('S1', 'C1'): 10,
 ('S1', 'C2'): 5,
 ('S1', 'C3'): 20,
 ('S1', 'C4'): 12,
 ('S2', 'C1'): 12,
 ('S2', 'C2'): 7,
 ('S2', 'C3'): 12,
 ('S2', 'C4'): 19,
 ('S3', 'C1'): 6,
 ('S3', 'C2'): 12,
 ('S3', 'C3'): 16,
 ('S3', 'C4'): 17}

In [9]:
# Create model instance
model = pyo.ConcreteModel()

In [10]:
# Sets for suppliers I and customers J
model.I = pyo.Set(initialize=availabilities.keys()) 
model.J = pyo.Set(initialize=demands.keys())

In [11]:
# Paramters
model.b = pyo.Param(model.I, initialize=availabilities)  # Supply
model.d = pyo.Param(model.J, initialize=demands)  # Demand
model.c = pyo.Param(model.I, model.J, initialize=costs)  # Cost


In [12]:
model.c.display()  # Display cost parameter

c : Size=12, Index=I*J, Domain=Any, Default=None, Mutable=False
    Key          : Value
    ('S1', 'C1') :    10
    ('S1', 'C2') :     5
    ('S1', 'C3') :    20
    ('S1', 'C4') :    12
    ('S2', 'C1') :    12
    ('S2', 'C2') :     7
    ('S2', 'C3') :    12
    ('S2', 'C4') :    19
    ('S3', 'C1') :     6
    ('S3', 'C2') :    12
    ('S3', 'C3') :    16
    ('S3', 'C4') :    17


In [13]:
# Decision variables
model.x = pyo.Var(model.I, model.J, within=pyo.NonNegativeReals)  # Amount shipped from I to J

In [14]:
# Supplier availablity constraints
def av_cstr(model, i):
    return sum(model.x[i,:]) <= model.b[i]


model.av_cstr = pyo.Constraint(model.I, rule=av_cstr)  # Apply to all suppliers


In [15]:
model.av_cstr.pprint()  # Display constraints

av_cstr : Size=3, Index=I, Active=True
    Key : Lower : Body                                      : Upper : Active
     S1 :  -Inf : x[S1,C1] + x[S1,C2] + x[S1,C3] + x[S1,C4] :  14.0 :   True
     S2 :  -Inf : x[S2,C1] + x[S2,C2] + x[S2,C3] + x[S2,C4] :  26.0 :   True
     S3 :  -Inf : x[S3,C1] + x[S3,C2] + x[S3,C3] + x[S3,C4] :  11.0 :   True


In [16]:
# Demand equality constraints
def dem_cstr(model, j):
    return sum(model.x[:,j]) == model.d[j]  # Demand must be met exactly

model.dem_cstr = pyo.Constraint(model.J, rule=dem_cstr)  # Apply to all customers

In [17]:
model.dem_cstr.pprint()  # Display constraints

dem_cstr : Size=4, Index=J, Active=True
    Key : Lower : Body                           : Upper : Active
     C1 :   5.0 : x[S1,C1] + x[S2,C1] + x[S3,C1] :   5.0 :   True
     C2 :  13.0 : x[S1,C2] + x[S2,C2] + x[S3,C2] :  13.0 :   True
     C3 :  15.0 : x[S1,C3] + x[S2,C3] + x[S3,C3] :  15.0 :   True
     C4 :  17.0 : x[S1,C4] + x[S2,C4] + x[S3,C4] :  17.0 :   True


In [18]:
# Objective function
def obj(model):
    total_cost = sum(model.c[i,j] * model.x[i,j] for i in model.I for j in model.J)
    return total_cost
   

model.obj = pyo.Objective(rule=obj, sense=pyo.minimize)  # Minimize total cost

In [19]:
model.obj.pprint()  # Display objective function

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 10*x[S1,C1] + 5*x[S1,C2] + 20*x[S1,C3] + 12*x[S1,C4] + 12*x[S2,C1] + 7*x[S2,C2] + 12*x[S2,C3] + 19*x[S2,C4] + 6*x[S3,C1] + 12*x[S3,C2] + 16*x[S3,C3] + 17*x[S3,C4]


In [20]:
# Instantiate Highs persistent solver (make sure highspy is installed)
solver = pyo.SolverFactory("appsi_highs")

In [21]:
# Apply method solve
solver.solve(model)

{'Problem': [{'Lower bound': 526.0, 'Upper bound': 526.0, 'Number of objectives': 1, 'Number of constraints': 0, 'Number of variables': 0, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Termination message': 'TerminationCondition.optimal'}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [22]:
# Use objective as a callable to see its value
model.obj()

526.0

In [26]:
model.x.extract_values()  # Extract values from decision variables

{('S1', 'C1'): 0.0,
 ('S1', 'C2'): 2.0,
 ('S1', 'C3'): 0.0,
 ('S1', 'C4'): 12.0,
 ('S2', 'C1'): 0.0,
 ('S2', 'C2'): 11.0,
 ('S2', 'C3'): 15.0,
 ('S2', 'C4'): 0.0,
 ('S3', 'C1'): 5.0,
 ('S3', 'C2'): 0.0,
 ('S3', 'C3'): 0.0,
 ('S3', 'C4'): 5.0}

In [23]:
sol = [
    {"from": i, "to": j, "value": val}
    for (i, j), val in model.x.extract_values().items()
]

In [24]:
dataframe = pd.DataFrame(sol).pivot(
    index="from", columns="to", values="value"
)

In [25]:
display(dataframe)

to,C1,C2,C3,C4
from,,,,
S1,0.0,2.0,0.0,12.0
S2,0.0,11.0,15.0,0.0
S3,5.0,0.0,0.0,5.0
